# <div align="center">Detail understanding of HMR paper</div>
---------------------------------------------------------------------

you can Find me on Github:
> ###### [ GitHub](https://github.com/lev1khachatryan)


HMR is an end-to-end framework for recovering a full 3D mesh of a human body from a single RGB image. It uses the ***generative human body*** model, ***SMPL***, which parameterizes the mesh by 3D joint angles and a low-dimensional linear shape space.

The output is holistic - always infer the full 3D body even in case of ***occlusion*** and ***truncation***.

In 3D analysis most approaches focus on recovering 3D joint locations which alone are not constrain the full DoF (degree of freedom) at each joint. This means that it is non-trivial to estimate the full pose of the body from only the 3D joint locations. 

In contrast, HMR outputs the ***relative 3D rotation matrices for each joint in the kinematic tree***, capturing information about 3D head and limb orientation. Predicting rotations also ensures that limbs are symmetric and of valid length. Model implicitly learns the joint angle limits from datasets of 3D body models.

## Model

During training we assume that all images are annotated with ground truth 2D joints. We also consider the case in which some have 3D annotations as well. Additionally we assume that there is a pool of 3D meshes of human bodies of varying shape and pose. Since these meshes do not necessarily have a corresponding image, we refer to this data as unpaired.

Figure 2 shows the overview of the proposed network architecture, which can be trained end-to-end. Convolutional features of the image are sent to the iterative 3D regression module whose objective is to infer the 3D human body and the camera such that its 3D joints project onto the annotated 2D joints. The inferred parameters are also sent to an adversarial discriminator network whose task is to determine if the 3D parameters are real meshes from the unpaired data. This encourages the network to output 3D human bodies that lie on the manifold of human bodies and acts as a weaksupervision for in-the-wild images without ground truth 3D annotations. Due to the rich representation of the 3D mesh model, this data-driven prior can capture joint angle limits, anthropometric constraints (e.g. height, weight, bone ratios), and subsumes the geometric priors used by models that only predict 3D joint locations. When ground truth 3D information is available, we may use it as an intermediate loss. In all, our overall objective is:

$L=\lambda(L_{reproj} + 1L_{3D}) + L_{adv}$

where $\lambda$ controls the relative importance of each objective, $1$ is an indicator function that is 1 if ground truth 3D is available for an image and 0 otherwise.

<img src='_assets/HMR/HMR.png'>

**Figure 2**: Overview of the proposed framework. An image I is passed through a convolutional encoder. This is sent to
an iterative 3D regression module that infers the latent 3D representation of the human that minimizes the joint reprojection
error. The 3D parameters are also sent to the discriminator D, whose goal is to tell if these parameters come from a real
human shape and pose.

## 3D Body Representation

We encode the 3D mesh of a human body using the Skinned Multi-Person Linear (SMPL) model. SMPL is a generative model that factors human bodies into ***shape*** (how individuals vary in height, weight, body proportions) and ***pose*** (how the 3D surface deforms with articulation).

The shape $\beta \in R^{10}$ is parameterized by the first 10 coefs of a PCA shape space. The pose $\theta \in R^{3K}$ is modeled by relative 3D rotation of K = 23 joints in axis-angle representation.

SMPL is a differentiable function that outputs a triangulated mesh with N = 6980 vertices, $M(\theta, \beta) \in R^{3xN}$, which is obtained by shaping the template body vertices conditioned on $\beta$ and $\theta$, then articulating the bones according to the joint rotations $\theta$ via ***forward kinematics***, and finally deforming the surface with ***linear blend skinning***. The 3D keypoints used for reprojection error, $X(\theta, \beta) \in R^{3xP}$, are obtained by linear regression from the final mesh vertices.

We employ the weak-perspective camera model and solve for the global rotation $R \in R^{3x3}$ in axis-angle representation, translation $t \in R^2$ and scale $s \in R$. Thus the set of parameters that represent the 3D reconstruction of a human body is expressed as a 85 dimensional vector $\Theta = \{ \theta, \beta, R, t, s \}$. Given $\Theta$, the projection of $X(\theta, \beta)$ is $\hat{x}=s \Pi(RX(\theta, \beta)) + t$  where $\Pi$ is an orthographic projection.

In [ ]:
verts, Js, pred_Rs = smpl(shapes, poses, get_skin=True)
pred_Rs = tf.reshape(pred_Rs, [-1, 24, 9])

print('Shape of vertices - verts: ', verts.shape)
print('Shape of Joints - Js: ', Js.shape)
print('Shape of rotation matrices of poses - pred_Rs: ', pred_Rs.shape)

# Shape of vertices - verts:  (1, 6890, 3)
# Shape of Joints - Js:  (1, 19, 3)
# Shape of rotation matrices of poses - pred_Rs:  (1, 24, 9)

In [ ]:
def batch_orth_proj_idrot(X, camera, name=None):
    """
    X is N x num_points x 3
    camera is N x 3
    same as applying orth_proj_idrot to each N 
    """
    with tf.name_scope(name, "batch_orth_proj_idrot", [X, camera]):
        
        # reshape to (1, 1, 3)
        camera = tf.reshape(camera, [-1, 1, 3], name="cam_adj_shape")

        # (1, 19, 2) + (1, 1, 2) = (1, 19, 2)
        X_trans = X[:, :, :2] + camera[:, :, 1:]

        shape = tf.shape(X_trans)
        
        # camera[:, :, 0] is (1, 1)
        # tf.reshape(X_trans, [shape[0], -1]) is (1, 38)
        # camera[:, :, 0] * tf.reshape(X_trans, [shape[0], -1]) is (1, 38)
        return tf.reshape(
            camera[:, :, 0] * tf.reshape(X_trans, [shape[0], -1]), shape)

In [ ]:
pred_kp = batch_orth_proj_idrot(Js, cams, name='proj2d_stage%d' % i)

print('Shape of camera - cams: ', cams.shape)
print('Shape of predicted key-points - pred_kp: ', pred_kp.shape)

# Shape of camera - cams:  (1, 3)
# Shape of predicted key-points - pred_kp:  (1, 19, 2)

## Iterative 3D Regression with Feedback

The goal of the 3D regression module is to output $\Theta$ given an image encoding $\phi$ such that the joint reprojection error $L_{reproj}=\sum_{i}||v_i(x_i - \hat{x_i})||_1$ is minimized. Here $v_i \in \{0, 1\}^K$ is the visibility for each of the K joints. However, directly regressing $\Theta$ in one go is a challenging task, particularly because $\Theta$ includes rotation parameters. In this work, we regress $\Theta$ in an iterative error feedback (IEF) loop, where progressive changes are made recurrently to the current estimate. 

Specifically, the 3D regression module takes the image features $\phi$ and the current parameters $\Theta_t$ as an input and outputs the residual $\Delta \Theta_t$. The parameter is updated by adding this residual to the current estimate $\Theta_{t+1} = \Theta_t + \Delta \Theta_t$. The initial estimate $\Theta_0$ is set as the mean $\hat{\Theta}$. In this work, we keep everything in the latent space and simply concatenate the features $[\phi, \Theta]$ as the input to the regressor. We find that
this works well and is suitable when differentiable rendering of the parameters is non-trivial.

Additional direct 3D supervision may be employed when paired ground truth 3D data is available. The most common form of 3D annotation is the 3D joints. Below are the definitions of the 3D losses.

$L_{3D}=L_{3D joints} + L_{3D smpl}$

$L_{joints}=||(X_i - \hat{X_i})||_2^2$

$L_{smpl}=||[\beta_i, \theta_i] - [\hat{\beta_i}, \hat{\theta_i}]||_2^2$

In [ ]:
print('Shape of encoded image - img_feat: ', img_feat.shape)
# Shape of encoded image - img_feat:  (1, 2048)

# loads mean params from neutral_smpl_mean_params.h5 pretrained model
theta_prev = load_mean_param()

print('Shape of initial Theta - theta_prev: ', theta_prev.shape)
# Shape of initial Theta - theta_prev:  (1, 85)

In [ ]:
# Main IEF loop, num_stage is by default 3
for i in np.arange(num_stage):
    # (1, 2133)
    state = tf.concat([img_feat, theta_prev], 1)
    
    # threed_enc_fn is 3 layered MLP (last is the output)
    delta_theta, _ = threed_enc_fn(
        state, num_output=85, reuse=True)

    # Compute new theta
    theta_here = theta_prev + delta_theta
    # cam = N x 3, pose N x self.num_theta, shape: N x 10
    cams = theta_here[:, :3]
    poses = theta_here[:, 3:(3 + 24*3)]
    shapes = theta_here[:, (3 + 24*3):]
    # pred_Rs is Nx24x3x3 rotation matrices of poses
    verts, Js, pred_Rs = self.smpl(shapes, poses, get_skin=True)
    pred_kp = batch_orth_proj_idrot(
        Js, cams, name='proj2d_stage%d' % i)
    # --- Compute losses: loss_kps is initially empty list
    loss_kps.append(keypoint_loss(kp_loader, pred_kp))
    pred_Rs = tf.reshape(pred_Rs, [-1, 24, 9])
    if self.use_3d_label:
        loss_poseshape, loss_joints = get_3d_loss(pred_Rs, shapes, Js)
        loss_3d_params.append(loss_poseshape)
        loss_3d_joints.append(loss_joints)

    # Save pred_rotations for Discriminator: 
    # fake_rotations and fake_shapes are initially empty lists
    fake_rotations.append(pred_Rs[:, 1:, :])
    fake_shapes.append(shapes)

    theta_prev = theta_here

In [ ]:
def keypoint_loss(kp_gt, kp_pred, scale=1., name=None):
    """
    computes: \Sum_i [0.5 * vis[i] * |kp_gt[i] - kp_pred[i]|] / (|vis|)
    Inputs:
      kp_gt  : N x K x 3
      kp_pred: N x K x 2
    """
    with tf.name_scope(name, "keypoint_l1_loss", [kp_gt, kp_pred]):
        kp_gt = tf.reshape(kp_gt, (-1, 3))
        kp_pred = tf.reshape(kp_pred, (-1, 2))

        vis = tf.expand_dims(tf.cast(kp_gt[:, 2], tf.float32), 1)
        res = tf.losses.absolute_difference(kp_gt[:, :2], kp_pred, weights=vis)
        return res
    
def compute_3d_loss(params_pred, params_gt, has_gt3d):
    """
    Computes the l2 loss between 3D params pred and gt for those data that has_gt3d is True.
    Parameters to compute loss over:
    3Djoints: 14*3 = 42
    rotations:(24*9)= 216
    shape: 10
    total input: 226 (gt SMPL params) or 42 (just joints)

    Inputs:
      params_pred: N x {226, 42}
      params_gt: N x {226, 42}
      # has_gt3d: (N,) bool
      has_gt3d: N x 1 tf.float32 of {0., 1.}
    """
    with tf.name_scope("3d_loss", [params_pred, params_gt, has_gt3d]):
        weights = tf.expand_dims(tf.cast(has_gt3d, tf.float32), 1)
        res = tf.losses.mean_squared_error(
            params_gt, params_pred, weights=weights) * 0.5
        return res
def get_3d_loss(Rs, shape, Js):
    """
    Rs is N x 24 x 3*3 rotation matrices of pose
    Shape is N x 10
    Js is N x 19 x 3 joints

    Ground truth:
    self.poseshape_loader is a long vector of:
       relative rotation (24*9)
       shape (10)
       3D joints (14*3)
    """
    Rs = tf.reshape(Rs, [self.batch_size, -1])
    params_pred = tf.concat([Rs, shape], 1, name="prep_params_pred")
    # 24*9+10 = 226
    gt_params = self.poseshape_loader[:, :226]
    loss_poseshape = self.e_3d_weight * compute_3d_loss(
        params_pred, gt_params, self.has_gt3d_smpl)
    # 14*3 = 42
    gt_joints = self.poseshape_loader[:, 226:]
    pred_joints = Js[:, :14, :]
    # Align the joints by pelvis.
    pred_joints = align_by_pelvis(pred_joints)
    pred_joints = tf.reshape(pred_joints, [self.batch_size, -1])
    gt_joints = tf.reshape(gt_joints, [self.batch_size, 14, 3])
    gt_joints = align_by_pelvis(gt_joints)
    gt_joints = tf.reshape(gt_joints, [self.batch_size, -1])

    loss_joints = self.e_3d_weight * compute_3d_loss(
        pred_joints, gt_joints, self.has_gt3d_joints)

    return loss_poseshape, loss_joints

## Factorized Adversarial Prior

The reprojection loss encourages the network to produce a 3D body that explains the 2D joint locations, however  nthropometrically implausible 3D bodies or bodies with gross self-intersections may still minimize the reprojection loss. To regularize this, we use a discriminator network D that is trained to tell whether SMPL parameters correspond to a real body or not. We refer to this as an adversarial prior since the discriminator acts as a data-driven prior that guides the 3D inference.

A further benefit of employing a rich, explicit 3D representation like SMPL is that we precisely know the meaning of the latent space. In particular SMPL has a factorized form that we can take advantage of to make the adversary more data efficient and stable to train. More concretely, we mirror the shape and pose decomposition of SMPL and train a discriminator for shape and pose independently. The pose is based on a kinematic tree, so we further decompose the pose discriminators and train one for each joint rotation. This amounts to learning the angle limits for each joint. In order to capture the joint distribution of the entire kinematic tree, we also learn a discriminator that takes in all the rotations.

Since the input to each discriminator is very low dimensional (10-D for $\beta$, 9-D for each joint and 9K-D for all joints), they can each be small networks, making them rather stable to train. All pose discriminators share a common feature space of rotation matrices and only the final classifiers are learned separately.

In all we train K + 2 discriminators. Each discriminator $D_{i}$ outputs values between [0, 1], representing the probability that $\Theta$ came from the data. In practice we use the least square formulation for its stability. Let E represent the
encoder including the image encoder and the 3D module.

Then the adversarial loss function for the encoder is: $min L_{adv}(E) \sum_{i}E_{\Theta \sim p_E}[(D_i(E(I))-1)^2]$

and the objective for each discriminator is $min L(D_i) = E_{\Theta \sim p_{data}}[(D_i(\Theta)-1)^2] + E_{\Theta \sim p_E}[D_i(E(I))^2]$

We optimize $E$ and all $D_{i}$'s jointly

In [ ]:
def setup_discriminator(self, fake_rotations, fake_shapes):
    # Compute the rotation matrices of "real" pose
    # These guys are in 24 x 3.
    real_rotations = batch_rodrigues(tf.reshape(self.pose_loader, [-1, 3]))
    real_rotations = tf.reshape(real_rotations, [-1, 24, 9])
    # Ignoring global rotation. N x 23*9
    # The # of real rotation is B*num_stage so it's balanced.
    real_rotations = real_rotations[:, 1:, :]
    all_fake_rotations = tf.reshape(
        tf.concat(fake_rotations, 0),
        [self.batch_size * self.num_stage, -1, 9])
    comb_rotations = tf.concat(
        [real_rotations, all_fake_rotations], 0, name="combined_pose")

    comb_rotations = tf.expand_dims(comb_rotations, 2)
    all_fake_shapes = tf.concat(fake_shapes, 0)
    comb_shapes = tf.concat(
        [self.shape_loader, all_fake_shapes], 0, name="combined_shape")

    disc_input = {
        'weight_decay': self.d_wd,
        'shapes': comb_shapes,
        'poses': comb_rotations
    }

    self.d_out, self.D_var = Discriminator_separable_rotations(
        **disc_input)

    self.d_out_real, self.d_out_fake = tf.split(self.d_out, 2)
    # Compute losses:
    with tf.name_scope("comp_d_loss"):
        self.d_loss_real = tf.reduce_mean(
            tf.reduce_sum((self.d_out_real - 1)**2, axis=1))
        self.d_loss_fake = tf.reduce_mean(
            tf.reduce_sum((self.d_out_fake)**2, axis=1))
        # Encoder loss
        self.e_loss_disc = tf.reduce_mean(
            tf.reduce_sum((self.d_out_fake - 1)**2, axis=1))

In [ ]:
def batch_rodrigues(theta, name=None):
    """
    Theta is N x 3
    """
    with tf.name_scope(name, "batch_rodrigues", [theta]):
        batch_size = theta.shape.as_list()[0]
        
        angle = tf.expand_dims(tf.norm(theta + 1e-8, axis=1), -1)
        r = tf.expand_dims(tf.div(theta, angle), -1)

        angle = tf.expand_dims(angle, -1)
        cos = tf.cos(angle)
        sin = tf.sin(angle)

        outer = tf.matmul(r, r, transpose_b=True, name="outer")

        eyes = tf.tile(tf.expand_dims(tf.eye(3), 0), [batch_size, 1, 1])
        R = cos * eyes + (1 - cos) * outer + sin * batch_skew(
            r, batch_size=batch_size)
        return R

In [ ]:
def Discriminator_separable_rotations(
        poses,
        shapes,
        weight_decay,
):
    """
    23 Discriminators on each joint + 1 for all joints + 1 for shape.
    To share the params on rotations, this treats the 23 rotation matrices
    as a "vertical image":
    Do 1x1 conv, then send off to 23 independent classifiers.

    Input:
    - poses: N x 23 x 1 x 9, NHWC ALWAYS!!
    - shapes: N x 10
    - weight_decay: float

    Outputs:
    - prediction: N x (1+23) or N x (1+23+1) if do_joint is on.
    - variables: tf variables
    """
    data_format = "NHWC"
    with tf.name_scope("Discriminator_sep_rotations", [poses, shapes]):
        with tf.variable_scope("D") as scope:
            with slim.arg_scope(
                [slim.conv2d, slim.fully_connected],
                    weights_regularizer=slim.l2_regularizer(weight_decay)):
                with slim.arg_scope([slim.conv2d], data_format=data_format):
                    poses = slim.conv2d(poses, 32, [1, 1], scope='D_conv1')
                    poses = slim.conv2d(poses, 32, [1, 1], scope='D_conv2')
                    theta_out = []
                    for i in range(0, 23):
                        theta_out.append(
                            slim.fully_connected(
                                poses[:, i, :, :],
                                1,
                                activation_fn=None,
                                scope="pose_out_j%d" % i))
                    theta_out_all = tf.squeeze(tf.stack(theta_out, axis=1))

                    # Do shape on it's own:
                    shapes = slim.stack(
                        shapes,
                        slim.fully_connected, [10, 5],
                        scope="shape_fc1")
                    shape_out = slim.fully_connected(
                        shapes, 1, activation_fn=None, scope="shape_final")
                    """ Compute joint correlation prior!"""
                    nz_feat = 1024
                    poses_all = slim.flatten(poses, scope='vectorize')
                    poses_all = slim.fully_connected(
                        poses_all, nz_feat, scope="D_alljoints_fc1")
                    poses_all = slim.fully_connected(
                        poses_all, nz_feat, scope="D_alljoints_fc2")
                    poses_all_out = slim.fully_connected(
                        poses_all,
                        1,
                        activation_fn=None,
                        scope="D_alljoints_out")
                    out = tf.concat([theta_out_all,
                                     poses_all_out, shape_out], 1)

            variables = tf.contrib.framework.get_variables(scope)
            return out, variables

## Implementation Details

All images are scaled to 224 × 224 preserving the aspect ratio such that the diagonal of the tight bounding box is roughly 150px. The images are randomly scaled, translated, and flipped. Mini-batch size is 64. When paired 3D supervision is employed each mini-batch is balanced such that it consists of half 2D and half 3D samples. All experiments use all datasets with paired 3D loss unless otherwise specified.

The definition of the K = 23 joints in SMPL do not align perfectly with the common joint definitions used by these datasets. We se a regressor to obtain he ***14*** joints of Human3.6M from the reconstructed mesh. In addition, we also incorporate the 5 face keypoints from the MS COCO dataset. New keypoints can easily be incorporated with the mesh representation by specifying the corresponding vertex IDs. In total the reprojection error is computed over P = 19 keypoints.

## Architecture

We use the ResNet-50 network for encoding the image, pretrained on the ImageNet classification task. The ResNet output is average pooled, producing features $\phi \in R^{2048} $. 

The 3D regression module consists of two fully-connected layers with 1024 neurons each with a dropout layer in between, followed by a final layer of 85D neurons. We use T = 3 iterations for all of our experiments. 

The discriminator ***for the shape*** is two fully-connected layers with 10, 5, and 1 neurons. For pose, $\theta$ is first converted to K many 3 × 3 rotation matrices via the ***Rodrigues formula***. Each rotation matrix is sent to a common embedding
network of two fully-connected layers with 32 hidden neurons. Then the outputs are sent to K = 23 different discriminators that output 1-D values. The discriminator for overall pose distribution concatenates all K ∗ 32 representations through another two fully-connected layers of 1024 neurons each and finally outputs a 1D value. All layers use ReLU activations except the final layer. The learning rates of the encoder and the discriminator network are set to 1 × 10−5 and 1×10−4 respectively. We use the Adam solver and train for 55 epochs.